In [ ]:
from astropy.io import fits
from astropy.table import unique, vstack
from astropy.table import Table
from astropy.time import Time
from astroquery.mast import Mast,Observations
import matplotlib.pyplot as plt
import os
import pandas as pd
import glob
import json
import jwst

In [ ]:
import astroquery

In [ ]:
astroquery.__version__

In [ ]:
jwst.__version__

In [ ]:
def set_params(parameters):
    return [{"paramName" : p, "values" : v} for p, v in parameters.items()]

def set_mjd_range(min, max):
    '''Set time range in MJD given limits expressed as ISO-8601 dates'''
    return {
        "min": Time(min, format='isot').mjd, 
        "max": Time(max, format='isot').mjd
        }

In [ ]:
def get_proposal_products(proposal_id:str):
    
    service = 'Mast.Jwst.Filtered.Nircam'

    keywords = {
            'exp_type':['NRC_CORON'],
            'instrume':['NIRCAM'],
            'date_obs_mjd': [set_mjd_range('2022-01-01','2023-05-03')],
               }

    params = {
            'columns': '*',
            'filters': set_params(keywords)
             }
    print(service, params)
    t = Mast.service_request(service, params)
    fn = list(set(t['filename']))
    ids = list(set(['_'.join(x.split('_')[:-1]) for x in fn]))
    
    matched_obs = Observations.query_criteria(instrument_name='Nircam',
                                         )
    batch_size = 100
    batches = [matched_obs[i:i+batch_size] for i in range(0, len(matched_obs), batch_size)]
    t = [Observations.get_product_list(obs) for obs in batches]
    products = unique(vstack(t), keys='productFilename')
    filtered_products = Observations.filter_products(products,
                                                 #dataproduct_type='image',
                                                 productType = ['SCIENCE'],
                                                 #proposal_id = [f'{proposal_id}'],
                                                 productSubGroupDescription = [#'UNCAL',
                                                                               #'RATEINTS',
                                                                               'CALINTS']
                                                 )
    
    return products,filtered_products
    

In [ ]:
PROPOSAL_ID = '1412'

In [ ]:
products,filtered_products = get_proposal_products(proposal_id=PROPOSAL_ID)

In [ ]:
len(filtered_products), len(products)

In [ ]:
#prd_df = filtered_products.to_pandas()

In [ ]:
def determine_and_count(column:str,table):    
    prop_list = []
    counter = {}
    for i in range(len(table['proposal_id'])):

        if table[f'{column}'][i] not in prop_list:

            prop_list.append(table[f'{column}'][i])
            try:
                counter[table[f'{column}'][i]] = 1
            except:
                #print(table[f'{column}'][i])
                continue
        else:
            try:
                counter[table[f'{column}'][i]] += 1
            except:
                continue
            
    #print(prop_list)
    print(counter)
    print(len(counter))

In [ ]:
determine_and_count('proposal_id',filtered_products)

In [ ]:
def get_json_table(table,suffix):
    
    testing_df = table.to_pandas()
    product_files = testing_df.loc[testing_df['productSubGroupDescription'] == f'{suffix}']['obs_id']
    obs_ids = []
    for i in product_files:
        obs_ids.append(i)
        
    products_df = testing_df.loc[testing_df['obs_id'].isin(obs_ids)]
    info_files = products_df.loc[products_df['productType']=='INFO']
    info_table = Table.from_pandas(info_files)
    
    return info_table

In [ ]:
def get_rateints_files(table):
    
    rateints_df = table.to_pandas()
    rateints_df = rateints_df.loc[rateints_df['productSubGroupDescription'] == 'RATEINTS']
    rateints_table = Table.from_pandas(rateints_df)
    
    return rateints_table

In [ ]:
filtered_products

# Downloading the Products

In [ ]:
#Observations.login(mast_token)

In [ ]:
coron_files = Observations.download_products(filtered_products,download_dir = f'/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/{PROPOSAL_ID}')

In [ ]:
def get_coron():
    
    path = '/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/CoronFiles/mastDownload/JWST/'
    coron_exptype = []
    noncoron_exptype = []

    for i in glob.glob(f'{path}/**/*.json'):

        with open(i,'r') as f:
            asn_file = json.load(f)

            fits_name = asn_file['products'][0]['members'][0]['expname']
            if 'nrc_coron' in asn_file['constraints']:
                #print('There is a coron right here')

                coron_exptype.append(fits_name)

            else:
                noncoron_exptype.append(fits_name)
                
    return coron_exptype, noncoron_exptype

In [ ]:
coron, noncoron = get_coron()

In [ ]:
coron

In [ ]:
def det_asn_type():
    
    path = '/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/CoronFiles/mastDownload/JWST/'
    image2 = []
    image3 = []

    for i in glob.glob(f'{path}/**/*.json'):

        with open(i,'r') as f:
            asn_file = json.load(f)

            fits_name = asn_file['asn_type']
            if 'image3' in asn_file['asn_type']:
                #print('There is a coron right here')
                image3.append(fits_name)
            else:
                image2.append(fits_name)
                
    return image3,image2

In [ ]:
image3, image2 = det_asn_type()

# Some Visualizations